<a href="https://colab.research.google.com/github/vdoffan/uda/blob/main/task6_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предложения доработок по заданию 6

**Примечание!** Код в данном ноутбуке не следует запускать. Данный формат оформления был выбран для повышения удобочитаемости и отдельно взятые ячейки с кодом не несут отдельный смысл - это просто предложения по улучшению программы.

## 1. Оптимизация структуры данных

Проблема: Использование списков для хранения лемм и связанных с ними данных (dict_lemmas, dict_count, dict_tonal) приводит к медленной работе при поиске и обновлении, особенно на больших объемах данных.

Решение: Перейти к использованию словарей (dict) для хранения лемм и их характеристик. Это позволит ускорить операции поиска, добавления и обновления данных благодаря постоянному времени доступа (O(1)).

### Реализация

1. Структура данных:

Вместо трех списков (dict_lemmas, dict_count, dict_tonal), использовать один словарь, где ключом будет лемма, а значением — объект, содержащий счетчик и тональность.

In [ ]:
from collections import defaultdict

class LemmaData:
    def __init__(self):
        self.count = 0
        self.tonality = 0.0

lemmas = defaultdict(LemmaData)

2. Обновление лемм:

При обработке каждого слова обновлять соответствующую запись в словаре.

In [ ]:
# Пример для слов без дефиса
lemma = spis2[j]
lemmas[lemma].count += 1
lemmas[lemma].tonality += korteg[1] if condition else 0  # Пример обновления тональности

3. Обработка слов с дефисом:

Аналогично обновлять словарь для составных слов.

In [ ]:
# Пример для слов с дефисом
lemma = spis[0].normal_form
lemmas[lemma].count += 1
lemmas[lemma].tonality += ton / len(words)

4. Вычисление общей тональности:

In [ ]:
t_g = sum(data.tonality * data.count for data in lemmas.values() if data.tonality > 0)
t_b = sum(-data.tonality * data.count for data in lemmas.values() if data.tonality < 0)
tonalities = t_g + t_b

5. Сортировка и подготовка данных для вывода:

In [ ]:
sorted_lemmas = sorted(lemmas.items(), key=lambda item: item[1].tonality * item[1].count, reverse=True)

**Преимущества:**

* Быстрый доступ: Поиск и обновление лемм осуществляется за постоянное время.
* Упрощение кода: Меньше циклов и проверок, код становится чище и понятнее.
* Меньше ошибок: Снижается вероятность ошибок при работе с несколькими списками параллельно.

## 2. Гибкость настроек

Можно реализовать такую функцию для обработки запросов пользователя.

In [ ]:
def get_user_input():
    """
    Запрашивает у пользователя необходимые параметры для анализа через терминал.

    Возвращает:
        dict: Словарь с введенными пользователем параметрами.
    """
    input_file = input("Введите имя входного файла (например, input.txt): ").strip()
    output_file = input("Введите имя выходного файла (по умолчанию 'output.csv'): ").strip()
    if not output_file:
        output_file = "output.csv"

    encoding = input("Введите кодировку входного файла (по умолчанию 'UTF-8'): ").strip()
    if not encoding:
        encoding = "UTF-8"

    save_plot = input("Хотите сохранить график? (да/нет): ").strip().lower()
    if save_plot == 'да':
        plot_file = input("Введите имя файла для сохранения графика (например, plot.png): ").strip()
    else:
        plot_file = None

    try:
        top_n = int(input("Введите количество топовых лемм для визуализации (по умолчанию 20): ").strip())
    except ValueError:
        top_n = 20

    try:
        genre_threshold = float(input("Введите порог для определения жанра (по умолчанию 0.05): ").strip())
    except ValueError:
        genre_threshold = 0.05

    return {
        'input_file': input_file,
        'output_file': output_file,
        'encoding': encoding,
        'save_plot': plot_file,
        'top_n': top_n,
        'genre_threshold': genre_threshold
    }

Пример использования
1. Запустите ноутбук.
2. Введите необходимые параметры по запросам:
* Имя входного файла (например, input.txt).
* Имя выходного файла (по умолчанию output.csv).
* Кодировку входного файла (по умолчанию UTF-8).
* Желаете ли сохранить график? (да или нет).
* Если да, введите имя файла для сохранения графика (например, plot.png).
* Количество топовых лемм для визуализации (по умолчанию 20).
* Порог для определения жанра (по умолчанию 0.05).
3. Результаты анализа будут выведены в терминал, сохранены в CSV-файл и, при необходимости, сохранен график.

С вводом можно работать следующим образом

In [ ]:
try:
    with open(args['input_file'], encoding=args['encoding']) as f:
        text = f.read().lower()
except FileNotFoundError:
    print(f"Файл {args['input_file']} не найден.")
    exit(1)
except UnicodeDecodeError:
    print(f"Невозможно декодировать файл {args['input_file']} с кодировкой {args['encoding']}.")
    exit(1)
except Exception as e:
    print(f"Ошибка при чтении файла: {e}")
    exit(1)

## 3. Расширение визуализации

Проблема: Программа отображает гистограмму, но не сохраняет её, что может быть неудобно для дальнейшего использования.

Решение: Добавить возможность сохранения графиков в файлы различных форматов (например, PNG, PDF, SVG) через аргументы командной строки.

### Реализация

Можно создать функцию для визуализации работы программы

In [ ]:
def visualize_top_lemmas(lemmas, top_n=20, save_path=None):
    sorted_lemmas = sorted(lemmas.items(), key=lambda item: abs(item[1]['tonality'] * item[1]['count']), reverse=True)
    sorted_lemmas = [item for item in sorted_lemmas if abs(item[1]['tonality']) >= 0.05][:top_n]
    sorted_lemmas.reverse()
    lemmas_names = [item[0] for item in sorted_lemmas]
    counts = [item[1]['count'] for item in sorted_lemmas]
    tonn = [item[1]['tonality'] for item in sorted_lemmas]
    colors = ['green' if t > 0 else 'red' for t in tonn]

    plt.figure(figsize=(10, 8))
    bars = plt.barh(lemmas_names, counts, color=colors)
    for bar, t in zip(bars, tonn):
        plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f"{t:.2f}", ha='left', va='center')
    plt.xlabel('Количество')
    plt.title('Слова, внесшие наибольший вклад в тональность текста')
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path)
        print(f"График сохранен как {save_path}")
    else:
        plt.show()


Передача аргументов в функцию визуализации

In [ ]:
args = get_user_input()
# ... предшествующий код ...

visualize_top_lemmas(lemmas, top_n=args.top_n, save_path=args.save_plot)

## Заключение

Надеюсь мои предложения помогут сделать программу лучше. Если есть какие-то замечания, всегда буду рад ответить и исправить их.